## 1. Import Packages

Below, we import both standard packages, and functions from the accompanying .py files

In [1]:
#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle

#Import metrics
from metrics import get_R2
from metrics import get_rho

#Import decoder functions
from decoders import KalmanFilterDecoder

/opt/anaconda/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release.  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 3: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5103)


## 2. Load Data

The data that we load is in the format described below. We have another example script, "neural_preprocessing.py" that may be helpful towards putting the data in this format.

Neural data should be a matrix of size "number of time bins" x "number of neurons", where each entry is the firing rate of a given neuron in a given time bin

The output you are decoding should be a matrix of size "number of time bins" x "number of features you are decoding"

In [2]:
# folder='/Users/jig289/Dropbox/MATLAB/Projects/In_Progress/BMI/Processed_Data/'
# folder='/Users/jig289/Dropbox/Grad_School/Research/Projects/In_Progress/Decoding/DataFiles/'
folder='/home/jglaser2/Data/DecData/'

with open(folder+'m1_test_data.pickle','rb') as f:
    neural_data,vels_binned,pos_binned,acc_binned=pickle.load(f)

## 3. Preprocess Data

### 3A. User Inputs
The user can define what time period to use spikes from (with respect to the output).

In [3]:
lag=-2 #What time bin of spikes should be used relative to the output
#(lag=-1 means use the spikes 1 bin before the output)

#-2 for m1, 0 for s1

### 3B. Format Covariates

#### Format Input

In [4]:
#The covariate is simply the matrix of firing rates for all neurons over time
X_kf=neural_data

#### Format Output

In [5]:
#For the Kalman filter, we use the position, velocity, and acceleration as outputs
#Ultimately, we are only concerned with the goodness of fit of velocity (for this dataset)
#But using them all as covariates helps performance

#We will now determine position
pos_binned=np.zeros(vels_binned.shape) #Initialize 
pos_binned[0,:]=0 #Assume starting position is at [0,0]
#Loop through time bins and determine positions based on the velocities
for i in range(pos_binned.shape[0]-1): 
    pos_binned[i+1,0]=pos_binned[i,0]+vels_binned[i,0]*.05 #Note that .05 is the length of the time bin
    pos_binned[i+1,1]=pos_binned[i,1]+vels_binned[i,1]*.05

#We will now determine acceleration    
temp=np.diff(vels_binned,axis=0) #The acceleration is the difference in velocities across time bins 
acc_binned=np.concatenate((temp,temp[-1:,:]),axis=0) #Assume acceleration at last time point is same as 2nd to last

#The final output covariates include position, velocity, and acceleration
y_kf=np.concatenate((pos_binned,vels_binned,acc_binned),axis=1)

### 3C. Process Covariates

#### Normalization and zero-centering

In [6]:
#Z-scoring function that works with Nans:
def zscore_nan(X,axis):
    X_zscore=(X - np.nanmean(X,axis=axis)) / np.nanstd(X,axis=axis)
    return X_zscore

In [7]:
#Normalize inputs
X_kf=zscore_nan(X_kf,axis=0)

#Zero-center outputs
y_kf_mean=np.mean(y_kf,axis=0)
y_kf=y_kf-y_kf_mean

#### Take lag into account

In [8]:
num_examples=X_kf.shape[0]

#Re-align data to take lag into account
if lag<0:
    y_kf=y_kf[-lag:,:]
    X_kf=X_kf[0:num_examples+lag,:]
if lag>0:
    y_kf=y_kf[0:num_examples-lag,:]
    X_kf=X_kf[lag:num_examples,:]

### 3D. Split into training/testing/validation sets
Note that parameters should be setting using a separate validation set. 
Then, the goodness of fit should be be tested on a testing set (separate from the training and validation sets).

### User Options

In [9]:
#Set what part of data should be part of the training/testing/validation sets
training_range=[0, 0.5]
testing_range=[0.5, 1]

### Split Data: For KF

In [10]:
#Number of examples after taking into account bins removed for lag alignment
num_examples_kf=X_kf.shape[0]
        
#Note that each range has a buffer of 1 bin at the beginning and end
#This makes it so that the different sets don't include overlapping data
training_set=np.arange(np.int(np.round(training_range[0]*num_examples_kf))+1,np.int(np.round(training_range[1]*num_examples_kf))-1)
testing_set=np.arange(np.int(np.round(testing_range[0]*num_examples_kf))+1,np.int(np.round(testing_range[1]*num_examples_kf))-1)

#Get training data
X_kf_train=X_kf[training_set,:]
y_kf_train=y_kf[training_set,:]

#Get testing data
X_kf_test=X_kf[testing_set,:]
y_kf_test=y_kf[testing_set,:]

## 4. Train Decoders


### Kalman Filter

In [11]:
#Declare model
model_kf=KalmanFilterDecoder()

#Fit model
model_kf.fit(X_kf_train,y_kf_train)

## 5. Get Fits on Test Data

In [12]:
num_tests=10
num_test_examples=y_kf_test.shape[0]
num_examples_per_test=np.round(np.divide(num_test_examples,num_tests))

#Initialize
mean_R2_kf=np.empty(num_tests)

for i in range(num_tests):

    idx=np.arange(num_examples_per_test*i,num_examples_per_test*(i+1))
    X_kf_test_temp=X_kf_test[idx,:]
    y_kf_test_temp=y_kf_test[idx,:]

    #Kalman Filter
    y_predicted_kf=model_kf.predict(X_kf_test_temp,y_kf_test_temp)
    mean_R2_kf[i]=np.mean(get_R2(y_kf_test_temp,y_predicted_kf)[2:4])

In [13]:
num_splits=10
num_bs=200
num_test_examples=y_test.shape[0]
num_examples_per_split=np.round(np.divide(num_test_examples,num_splits))

#Initialize
mean_R2_kf=np.empty(num_bs)

y_preds_kf=np.empty([num_splits,num_examples_per_split,y_test.shape[1]])

for i in range(num_splits):
    idx=np.arange(num_examples_per_split*i,num_examples_per_split*(i+1))
    X_test_temp=X_test[idx,:,:]
    X_flat_test_temp=X_flat_test[idx,:]
    
    #Kalman Filter
    y_preds_kf[i,:,:]=model_kf.predict(X_kf_test_temp,y_kf_test_temp)

    
y_test_bs=np.empty([num_splits,num_examples_per_split,y_test.shape[1]])
for i in range(num_splits):
    idx=np.arange(num_examples_per_split*i,num_examples_per_split*(i+1))
    y_test_bs[i,:,:]=y_test[idx,:]    
    
for i in range(num_bs): 
    
    random_idxs=np.floor(num_splits*np.random.rand(10)).astype(int)
    y_test_temp=np.reshape(y_test_bs[random_idxs,:,:],[y_test_bs.shape[0]*y_test_bs.shape[1],y_test_bs.shape[2]])    
    y_predicted_kf=np.reshape(y_preds_kf[random_idxs,:,:],[y_test_bs.shape[0]*y_test_bs.shape[1],y_test_bs.shape[2]])    
    
    mean_R2_kf[i]=np.mean(get_R2(y_kf_test_temp,y_predicted_kf)[2:4])


NameError: name 'y_test' is not defined

In [ ]:
mean_R2_kf

In [ ]:
np.mean(mean_R2_kf)

In [38]:
save_folder='/home/jglaser2/Data/Decoding_Results/'
with open(save_folder+'m1_results_kf.pickle','wb') as f:
    pickle.dump([mean_R2_kf],f)